In [18]:
import re
import pandas as pd
from tabulate import tabulate
from fetch_process_function import fetch_and_save_erc20_transfers, process_transactions, calculate_pnl, process_and_save_address
'''
data_process.ipynb和data_fetch.ipynb同样功能，但是通过封装函数实现了给日期地址等参数就可以返回数据，更加简洁
添加了pnl的计算，我们这里只计算第一种交易的pnl，其余的暂时忽略不计。
最后对于address也进行一个处理
'''
# ADDRESS = "0x5be9a4959308A0D0c7bC0870E319314d8D957dBB"
ADDRESS2 = "0x2c89a2ee92b9870f55989b4132a58c0e85222d86"
ADDRESS3 = "0x6c2a355929ee1262305e385ad49b84fe5f5a4777"

START_DATE = "2024-10-25 08:00" # Singapore Time，the function will convert it to UTC
END_DATE = "2024-11-28 08:00"
# OUTPUT_FILE = "C:/Users/YuweiCao/Documents/GitHub/Project/Project/etherscan/result"
OUTPUT_FILE2 = r"C:\Users\HUAWEI\Documents\GitHub\Project\etherscan\address2"
OUTPUT_FILE3 = r"C:\Users\HUAWEI\Documents\GitHub\Project\etherscan\address3"

API_KEY = "VQAIR728IM4Z8RZKPYBR4ESM5I3WBZK2C1"
BASE_TOKENS = {"USDT", "USDC", "USDE", "DAI"}
base_pattern = re.compile('|'.join(BASE_TOKENS))

In [19]:
fetch_and_save_erc20_transfers(
        address=ADDRESS2,
        start_date=START_DATE,
        end_date=END_DATE,
        output_file=OUTPUT_FILE2,
        offset=1000,
    )

transfers_df2 = pd.read_csv(OUTPUT_FILE2 + "/erc20_transfers.csv")
record_df_1_2, _, _ = process_transactions(transfers_df2, OUTPUT_FILE2, ADDRESS2, base_pattern)
fetch_and_save_erc20_transfers(
        address=ADDRESS3,
        start_date=START_DATE,
        end_date=END_DATE,
        output_file=OUTPUT_FILE3,
        offset=1000,
    )
transfers_df3 = pd.read_csv(OUTPUT_FILE3 + "/erc20_transfers.csv")
record_df_1_3, _, _  = process_transactions(transfers_df3, OUTPUT_FILE3, ADDRESS3, base_pattern)


2025-01-17 16:50:55,150 - INFO - Start block: 21038905, End block: 21282523
2025-01-17 16:50:55,153 - INFO - Fetching ERC20 transfers from block 21038905 to 21282523...
2025-01-17 16:50:56,827 - INFO - Page 1: Retrieved 596 transactions.
2025-01-17 16:50:57,809 - WARNING - API request failed on page 2: Request failed: No transactions found
2025-01-17 16:50:57,825 - INFO - Final dataset contains 596 unique transactions.
2025-01-17 16:50:57,956 - INFO - Data successfully saved to C:\Users\HUAWEI\Documents\GitHub\Project\etherscan\address2\erc20_transfers.csv
2025-01-17 16:50:57,960 - INFO - Transfers saved to C:\Users\HUAWEI\Documents\GitHub\Project\etherscan\address2


...Merging 212 groups of transactions...
Transaction Hash 0x2b198f8b99cc4a183fdb5125b09e1f32521b7f5351e6b12307d5022ba030708c contains 4 records:
Transaction Hash 0xe7965c51c772e31d0195646c27b1eda26ac069483f2e49db737cc8cb94fd3369 contains 4 records:
Transaction Hash 0xeb47a72d88c47d28a4150b339cab0bf693cc39a360f021743785a94102be0ddd contains 6 records:


2025-01-17 16:51:00,893 - INFO - Start block: 21038905, End block: 21282523
2025-01-17 16:51:00,897 - INFO - Fetching ERC20 transfers from block 21038905 to 21282523...
2025-01-17 16:51:02,553 - INFO - Page 1: Retrieved 1000 transactions.
2025-01-17 16:51:04,181 - INFO - Page 2: Retrieved 1000 transactions.
2025-01-17 16:51:05,762 - INFO - Page 3: Retrieved 474 transactions.
2025-01-17 16:51:06,774 - WARNING - API request failed on page 4: Request failed: No transactions found
2025-01-17 16:51:06,809 - INFO - Final dataset contains 2474 unique transactions.
2025-01-17 16:51:07,093 - INFO - Data successfully saved to C:\Users\HUAWEI\Documents\GitHub\Project\etherscan\address3\erc20_transfers.csv
2025-01-17 16:51:07,097 - INFO - Transfers saved to C:\Users\HUAWEI\Documents\GitHub\Project\etherscan\address3


...Merging 918 groups of transactions...
Transaction Hash 0x052ca792656135f686258d095a1cccb03c9b1d8ad6110a890bb84ad8c7e18d13 contains 4 records:
Transaction Hash 0x06b5ecfb93d25c3d81ebeee11cfd41d1647e8b5aa9b1f3ccd7a435996784ea61 contains 4 records:
Transaction Hash 0x09c275b15ecc39e71b98e336bcef59546d008bd602fb673f8de53415063283b5 contains 4 records:
Transaction Hash 0x0e6e0c9402d39ffed6e0fac563abc8a1990e34f50296958b0f4d2983c817681d contains 4 records:
Transaction Hash 0x1338ed70de802a70967c6ab2bad8b254c001f3d80680242680dcc07aeb68ded4 contains 4 records:
Transaction Hash 0x1437c9cf8bd31aa1aef8e427c918e513f143258ad7425c935086f39b6082078a contains 10 records:
Transaction Hash 0x22d64dc7cd02361e023c8aa05b7cd761cb483ab4e66f87234240cf2ccb77162d contains 8 records:
Transaction Hash 0x230939d9187c3a932b7a077bd6a9927dfd8e6530c5344fd82f3d4715a0a7c9b2 contains 4 records:
Transaction Hash 0x2cf69cdc1723babb332544cb41e33988e87b678be193d453ef2457d34e1e3b96 contains 6 records:
Transaction Hash 0x3a8

In [ ]:
# 计算pnl
def process_pnl(record_df):
    if not record_df.empty:
        unique_tokens_count = record_df["token_symbol"].nunique()
        unique_tokens = record_df["token_symbol"].unique()
        results = []
        pnl_dfs = []  # 用于存储每个token的pnl_df
        # 在USDC USDE USDT DAI的价格默认为1：1的情况下，不考虑不同的稳定币作为单位的情况

        for token in unique_tokens:
            token_df = record_df[record_df["formatted_record"].str.contains(fr"\b{token}\b", regex=True)]
            unique_price_units = token_df["price_unit"].unique()
            price_units_str = ", ".join(unique_price_units)
            
            print(f"{token} (Price Units: {price_units_str}): {len(token_df)} records")

            pnl_df = calculate_pnl(token_df)
            print(tabulate(pnl_df, headers='keys', tablefmt='grid', showindex=False, floatfmt=".2f"))   
            total_pnl = pnl_df["pnl"].sum()

            pnl_dfs.append(pnl_df)  # 将每个token的pnl_df存储到列表中
            results.append({"token": token, "price_unit": price_units_str, "total_pnl": total_pnl})

        pnl_summary_df = pd.DataFrame(results)
        print(tabulate(pnl_summary_df, headers='keys', tablefmt='grid', showindex=False, floatfmt=".2f"))
        
        # 将结果保存为 CSV 文件
        pnl_summary_df.to_csv(f'pnl_summary.csv', index=False)
        print("PNL summary saved to pnl_summary.csv")

        # 如果需要将每个 token 的 PNL 详情也保存为 CSV
        for token in unique_tokens:
            token_df = record_df[record_df["formatted_record"].str.contains(fr"\b{token}\b", regex=True)]
            pnl_df = calculate_pnl(token_df)
            pnl_df.to_csv(f"pnl_details_{token}.csv", index=False)
            print(f"PNL details for {token} saved to pnl_details_{token}.csv")
    else:
        print("No transactions found")


combined_record_df = pd.concat([record_df_1_2, record_df_1_3], ignore_index=True)
process_pnl(combined_record_df)


aEthWETH (Price Units: USDT/aEthWETH, aEthUSDT/aEthWETH): 522 records
+---------------------+-------------+--------+----------+---------+------------+----------------+
| Date                |   timeStamp | Type   |   amount |   price |        pnl |   net_position |
+=====================+=============+========+==========+=========+============+================+
| 2024-10-31 14:22:47 |  1730384567 | SELL   |  1117.94 | 2575.91 | -625032.63 |       -1117.94 |
+---------------------+-------------+--------+----------+---------+------------+----------------+
| 2024-10-31 14:28:59 |  1730384939 | BUY    |   224.21 | 2568.70 |  126973.86 |        -893.72 |
+---------------------+-------------+--------+----------+---------+------------+----------------+
| 2024-10-31 14:30:47 |  1730385047 | BUY    |   896.31 | 2570.26 |  506188.10 |           2.59 |
+---------------------+-------------+--------+----------+---------+------------+----------------+
| 2024-10-31 14:35:47 |  1730385347 | SELL   |  